In [11]:
from pathlib import Path

from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.embeddings import OpenAIEmbeddings


from src.constants import (
    NEO4J_URI,
    NEO4J_USERNAME,
    NEO4J_PASSWORD,
    OPENAI_API_KEY,
)

In [3]:
graph = Neo4jGraph(
    NEO4J_URI,
    NEO4J_USERNAME,
    NEO4J_PASSWORD,
)

In [5]:
llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [12]:
embeddings = OpenAIEmbeddings()

In [13]:
# グラフからノード検索用インデックスを取得
index = Neo4jVector.from_existing_graph(
    embedding=embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    # database=NEO4J_DATABASE,
    node_label="Person", # 検索対象ノード
    text_node_properties=["id", "text"], # 検索対象プロパティ
    embedding_node_property="embedding", # ベクトルデータの保存先プロパティ
    index_name="vector_index", # ベクトル検索用のインデックス名
    keyword_index_name="person_index", # 全文検索用のインデックス名
    search_type="hybrid" # 検索タイプに「ハイブリッド」を設定（デフォルトは「ベクター」）
)

In [14]:
query = "エレン"
docs_with_score = index.similarity_search_with_score(query, k=3)
docs_with_score

[(Document(page_content='\nid: エレン\ntext: '), 1.0),
 (Document(page_content='\nid: エレン・イェーガー\ntext: '), 0.9852305838006093),
 (Document(page_content='\nid: エレン・クルーガー\ntext: '), 0.9812929429273788)]

In [15]:
index2 = Neo4jVector.from_existing_relationship_index(
    embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    # database=NEO4J_DATABASE,
    index_name="relationship_vector_index" # ベクトル検索用のインデックス名
)

ValueError: The specified vector index name does not exist. Make sure to check if you spelled it correctly